In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from abc import ABC, abstractmethod
from collections import ChainMap
from functools import wraps, update_wrapper
import inspect
from inspect import Parameter, getsource, getsourcelines, getsourcefile, \
    getfile, ismodule, ismethod, isfunction

from htools import *

In [3]:
def foo(a, b=3):
    return 'foo', {'a': a, 'b': b}


def baz(c, d=6):
    return 'baz', {'c': c, 'd': d}


@add_kwargs(foo, baz, required=True)
def cli(**kwargs):
    print('in __call__', cli.__signature__, kwargs)
    f_res = foo(a=a, b=b)
    print(f_res)
    b_res = baz(c=c, d=d)
    print(b_res)

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
cli('ccc', 'aaa', b={2: 4})

in __call__ (c, a, d=6, b=3, **kwargs) {'c': 'ccc', 'd': 6, 'a': 'aaa', 'b': {2: 4}}


NameError: name 'a' is not defined

In [140]:
class CLI(ABC):
    
    def __init__(self, *funcs):
        self.funcs = list(funcs)
        
        @add_kwargs(*self.funcs, positional=True)
        def _call_wrapper(self, **kwargs):
            print(_call_wrapper.__signature__)
            print(Args(a=a, b=b, c=c, d=d))
            return self.call(**kwargs)
        CLI.__call__ = _call_wrapper
        
    @abstractmethod
    def call(self):
        raise NotImplementedError()

In [148]:
class MyCLI(CLI):
    
    # TODO: mostly works but I don't think we support args/kwargs at the 
    # moment. Maybe add them to signature with func name, i.e. bar_kwargs?
    def call(self, **kwargs):
        print('in __call__', kwargs)
        f_res = foo(a=a, b=b)
        b_res = baz(c=c, d=d)
        print(f_res)
        print(b_res)

In [149]:
def foo(a, b=3, **kwargs):
    return 'foo', {'a': a, 'b': b}

def baz(c, *args, d=6):
    return 'baz', {'c': c, 'd': d}

In [150]:
cli = MyCLI(foo, baz)

In [151]:
cli('c0', 'a1', 'd2')

in ctx {'c': 'c0', 'baz_args': 'd2', 'd': 6, 'a': 'a1', 'b': 3, 'foo_kwargs': {}, 'self': <__main__.MyCLI object at 0x107e7d7b8>}
(self, c, a, baz_args=(), b=3, *, d=6, foo_kwargs={}, **kwargs)
Args(a='a1', b=3, c='c0', d=6)
in __call__ {'c': 'c0', 'baz_args': 'd2', 'd': 6, 'a': 'a1', 'b': 3, 'foo_kwargs': {}}
('foo', {'a': 'a1', 'b': 3})
('baz', {'c': 'c0', 'd': 6})


In [110]:
cli = CLI(foo, baz)

In [111]:
cli.bar('x1', 9, d=11, b=44)

in ctx {'c': 'x1', 'd': 11, 'a': 9, 'b': 44, 'self': <__main__.CLI object at 0x1a16171c50>}
(self, c, a, d=6, b=3, **kwargs)
Args(a=9, b=44, c='x1', d=11)
in __call__ {'c': 'x1', 'd': 11, 'a': 9, 'b': 44}
396
x1x1x1x1x1x1x1x1x1x1x1


### Alternate strategies

- Update signature automatically with deco as above, but then explicitly update globals with kwargs. Is this any safer? Maybe not. I guess it removes 1 layer of hackery.
- 

## Picklable

In [10]:
class Picklable:
    
    def test(self):
        return getsource(type(self))

In [11]:
inspect.getmodule(Picklable)

<module '__main__'>

In [13]:
inspect.findsource??

In [165]:
getsource(Picklable)

TypeError: <module '__main__'> is a built-in class

In [164]:
Picklable().test()

TypeError: <module '__main__'> is a built-in class